# Bibliotecas
Importação das bibliotecas que serão utilizadas durante esse projeto.

In [188]:
import pandas as pd
import numpy as np

# Importando os dados
Essa seção serve para importação dos dados que serão utilizados nesse projeto.

In [189]:
df_gols = pd.read_csv('campeonato-brasileiro-gols.csv')
df_full = pd.read_csv('campeonato-brasileiro-full.csv')
# df_cartoes = pd.read_csv('campeonato-brasileiro-cartoes.csv')
df_estatisticas = pd.read_csv('campeonato-brasileiro-estatisticas-full.csv')

# df_gols.head()
# df_full.head()
# df_cartoes.head() 
df_estatisticas.head()    

,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios
0,1,1,Vasco,0,0,NaN,0,NaN,0,0,0,0,0
1,1,1,Guarani,0,0,NaN,0,NaN,0,0,0,0,0
2,2,1,Athletico-PR,0,0,NaN,0,NaN,0,0,0,0,0
3,2,1,Gremio,0,0,NaN,0,NaN,0,0,0,0,0
4,3,1,Flamengo,0,0,NaN,0,NaN,0,0,0,0,0


# Filtrando dados do Mirassol

In [190]:
df_filtrado_gols = df_gols[df_gols["clube"] == "Mirassol"].copy()
df_filtrado_full = df_full[(df_full["visitante"] == "Mirassol") | (df_full["mandante"] == "Mirassol")].copy()
# df_filtrado_cartoes = df_cartoes[df_cartoes["clube"] == "Mirassol"].copy()
df_filtrado_estatisticas = df_estatisticas[df_estatisticas["clube"] == "Mirassol"].copy()

# df_filtrado_gols.head()
# df_filtrado_full.head()
# df_filtrado_cartoes.head()
# df_filtrado_estatisticas.head()

df_partidas_mirassol = df_full[
    (df_full["mandante"] == "Mirassol") |
    (df_full["visitante"] == "Mirassol")
]

ids_partidas = df_partidas_mirassol["ID"].unique() 

df_estatisticas_adversarios = df_estatisticas[df_estatisticas["partida_id"].isin(ids_partidas)]

df_estatisticas_adversarios = df_estatisticas_adversarios[df_estatisticas_adversarios["clube"] != "Mirassol"]      

df_estatisticas_adversarios.head()    

df_gols_adversarios = df_gols[df_gols["partida_id"].isin(ids_partidas)].copy()

df_gols_adversarios = df_gols_adversarios[df_gols_adversarios["clube"] != "Mirassol"]
df_gols_adversarios.head()  


,partida_id,rodata,clube,atleta,minuto,tipo_de_gol
9865,8790,1,Cruzeiro,Gabriel Barbosa,22,NaN
9867,8790,1,Cruzeiro,Dudu,14,NaN
9893,8802,2,Fortaleza,Emmanuel Martínez,15,NaN
9918,8810,3,Bahia,Erick Pulga,40,NaN
9938,8819,4,Gremio,Martin Braithwaite,82,NaN


## Tratamento de NaN values

In [191]:
df_filtrado_gols["tipo_de_gol"] = df_filtrado_gols["tipo_de_gol"].fillna("Não Informado")

df_filtrado_gols.head()

df_gols_adversarios["tipo_de_gol"] = df_gols_adversarios["tipo_de_gol"].fillna("Não Informado") 

df_gols_adversarios.head()

,partida_id,rodata,clube,atleta,minuto,tipo_de_gol
9865,8790,1,Cruzeiro,Gabriel Barbosa,22,Não Informado
9867,8790,1,Cruzeiro,Dudu,14,Não Informado
9893,8802,2,Fortaleza,Emmanuel Martínez,15,Não Informado
9918,8810,3,Bahia,Erick Pulga,40,Não Informado
9938,8819,4,Gremio,Martin Braithwaite,82,Não Informado


## Tempo dos gols

In [192]:
def definir_periodo_gol(minuto):
    minuto_str = str(minuto)
    
    # Acréscimos 1º tempo
    if "45+" in minuto_str:
        return "Acréscimos 1º tempo"
    
    # Acréscimos 2º tempo
    if "90+" in minuto_str:
        return "Acréscimos 2º tempo"
    
    # Minutos normais
    try:
        minuto_int = int(minuto_str)
    except:
        return "Não informado"
    
    if 0 <= minuto_int <= 10:
        return "1-10"
    elif 11 <= minuto_int <= 20:
        return "11-20"
    elif 21 <= minuto_int <= 30:
        return "21-30"
    elif 31 <= minuto_int <= 40:
        return "31-40"
    elif 41 <= minuto_int <= 50:
        return "41-50"
    elif 51 <= minuto_int <= 60:
        return "51-60"
    elif 61 <= minuto_int <= 70:
        return "61-70"
    elif 70 <= minuto_int <= 80:
        return "71-80"
    elif 80 <= minuto_int <= 90:
        return "81-90"
    else:
        return "Fora do intervalo"



df_filtrado_gols["Período"] = df_filtrado_gols["minuto"].apply(definir_periodo_gol)

df_filtrado_gols.head()

df_gols_adversarios["Período"] = df_gols_adversarios["minuto"].apply(definir_periodo_gol)  

df_gols_adversarios.head()  

,partida_id,rodata,clube,atleta,minuto,tipo_de_gol,Período
9865,8790,1,Cruzeiro,Gabriel Barbosa,22,Não Informado,21-30
9867,8790,1,Cruzeiro,Dudu,14,Não Informado,11-20
9893,8802,2,Fortaleza,Emmanuel Martínez,15,Não Informado,11-20
9918,8810,3,Bahia,Erick Pulga,40,Não Informado,31-40
9938,8819,4,Gremio,Martin Braithwaite,82,Não Informado,81-90


In [193]:
def classificar_tempo(minuto):
    minuto_base = int(str(minuto).split("+")[0])
    
    if minuto_base <= 45:
        return "1º Tempo"
    else:
        return "2º Tempo"

df_filtrado_gols["Tempo"] = df_filtrado_gols["minuto"].apply(classificar_tempo)

df_filtrado_gols.head()

df_gols_adversarios["Tempo"] = df_gols_adversarios["minuto"].apply(classificar_tempo)  

df_gols_adversarios.head()

,partida_id,rodata,clube,atleta,minuto,tipo_de_gol,Período,Tempo
9865,8790,1,Cruzeiro,Gabriel Barbosa,22,Não Informado,21-30,1º Tempo
9867,8790,1,Cruzeiro,Dudu,14,Não Informado,11-20,1º Tempo
9893,8802,2,Fortaleza,Emmanuel Martínez,15,Não Informado,11-20,1º Tempo
9918,8810,3,Bahia,Erick Pulga,40,Não Informado,31-40,1º Tempo
9938,8819,4,Gremio,Martin Braithwaite,82,Não Informado,81-90,2º Tempo


## Resultado da partida

In [194]:
# df_filtrado_full["vencedor"] = df_filtrado_full["vencedor"].replace("-", "Empate")
df_filtrado_full.loc[df_filtrado_full["vencedor"] == "-", "vencedor"] = "Empate"

# df_filtrado_full.head(20)


df_filtrado_full["resultado"] = np.select([df_filtrado_full["vencedor"] == "Mirassol", 
                                           df_filtrado_full["vencedor"] == "Empate"], ["Vitória", "Empate"], default="Derrota")

df_filtrado_full["localidade"] = np.where(df_filtrado_full["mandante"] == "Mirassol", "Casa", "Fora")

df_filtrado_full["adversario"] = np.where(df_filtrado_full["mandante"] == "Mirassol", 
                                          df_filtrado_full['visitante'], df_filtrado_full['mandante']
                                                    )

## Criando métrica para descobrir quantas faltas foram necessárias para receber um tipo de cartão
Pode ser tanto colocado um NaN como 0 quando não houve cartões vermelhos, mas eu quero deixar como 0 para facilitar a criação do Dashboards.

In [195]:
df_filtrado_estatisticas["faltas_por_amarelo"] = np.where(
    df_filtrado_estatisticas["cartao_amarelo"] > 0,
    (df_filtrado_estatisticas["faltas"] / df_filtrado_estatisticas["cartao_amarelo"]).round(2),
    0
)

df_filtrado_estatisticas["faltas_por_vermelho"] = np.where(
    df_filtrado_estatisticas["cartao_vermelho"] > 0,
    (df_filtrado_estatisticas["faltas"] / df_filtrado_estatisticas["cartao_vermelho"]).round(2),
    0
)

## Renomeando coluna rodata -> rodada

In [196]:
df_filtrado_gols.rename(columns={"rodata": "rodada"}, inplace=True)
df_filtrado_estatisticas.rename(columns={"rodata": "rodada"}, inplace=True)
df_filtrado_full.rename(columns={"rodata": "rodada"}, inplace=True)
df_estatisticas_adversarios.rename(columns={"rodata": "rodada"}, inplace=True)

# Salvando como CSV

In [197]:
df_filtrado_gols.to_csv("gols_mirassol_filtrado.csv", index=False, decimal=",")
df_filtrado_full.to_csv("full_mirassol_filtrado.csv", index=False, decimal=",")
df_filtrado_estatisticas.to_csv("estatisticas_mirassol_filtrado.csv", index=False, decimal=",")
df_estatisticas_adversarios.to_csv("estatisticas_adversarios_mirassol.csv", index=False, decimal=",")  
df_gols_adversarios.to_csv("gols_adversarios_mirassol.csv", index=False, decimal=",") 